In [11]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import re
import numpy as np

In [23]:
df = pd.read_csv(r'/home/sheid/Project/pslvis/dataset/Recidivism.csv')
df = df.drop(columns=['Fiscal Year Released', 'Recidivism Reporting Year'])
X_ = df.iloc[:,:-1]
y_ = df.iloc[:,-1]

In [77]:
X_.columns

Index(['Race - Ethnicity', 'Age At Release ',
       'Convicting Offense Classification', 'Convicting Offense Type',
       'Convicting Offense Subtype', 'Main Supervising District',
       'Release Type', 'Release type: Paroled to Detainder united',
       'Part of Target Population'],
      dtype='object')

In [78]:
def create_transformer(cond):
    return FunctionTransformer(lambda x: np.where(cond(x), 1, 0), feature_names_out=lambda _:None)



ct = ColumnTransformer(
    [
        ('Person of Color', create_transformer(lambda x: x != "White - Non-Hispanic") , ['Race - Ethnicity']),
        ('Verbrechen (nicht Vergehen)', FunctionTransformer(lambda x: x.map(lambda x_: "Felony" in x_)) , ['Convicting Offense Classification']),
        ("Drogendelikt", create_transformer(lambda x: x == "Drug"), ["Convicting Offense Type"]),
        ("Eigentumsdelikt", create_transformer(lambda x: x == "Property"), ["Convicting Offense Type"]),
        ("Gewaltdelikt", create_transformer(lambda x: x == "Violent"), ["Convicting Offense Type"]),
        ("Alter", FunctionTransformer(lambda x: x.map(lambda x_: np.nan_to_num(np.array([int(e) for e in re.findall("\\d+", str(x_))]).mean(), nan=39).astype(int))), ["Age At Release "]),
        ("Vorzeitige Entlassung", create_transformer(lambda x:  x != "Parole"), ["Release Type"]),
    ]
)

ct.set_output(transform='pandas')
X = ct.fit_transform(X_)
X.columns = [x.split("__")[0] for x in X.columns]
X.columns

/tmp/ipykernel_30856/2976875330.py:13: RuntimeWarning: Mean of empty slice.
  ("Alter", FunctionTransformer(lambda x: x.map(lambda x_: np.nan_to_num(np.array([int(e) for e in re.findall("\\d+", str(x_))]).mean(), nan=39).astype(int))), ["Age At Release "]),
/home/sheid/.cache/pypoetry/virtualenvs/pslvis-ssQ-I2no-py3.12/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Index(['Person of Color', 'Verbrechen (nicht Vergehen)', 'Drogendelikt',
       'Eigentumsdelikt', 'Gewaltdelikt', 'Alter', 'Vorzeitige Entlassung'],
      dtype='object')

In [80]:
X.insert(0,"Rückfällig", y_)
X.to_csv("Recidivism_clean.csv", index=False)